# Important: do Cell > All Outputs > Clear before commiting

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import qiskit
from qiskit import BasicAer, QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.aqua.circuits import FixedValueComparator

from qiskit.transpiler.passes import Unroller
from qiskit.converters import circuit_to_dag, dag_to_circuit
import re 
import time


In [7]:
NUM_BITS = 5
S =  list(range(10, 20))
N =  len(S)
simulator = BasicAer.get_backend('statevector_simulator')
backend = BasicAer.get_backend('statevector_simulator')

In [3]:
x = QuantumRegister(NUM_BITS, name="xs") #name cannot be x or the conversion to qasm string and back will FAIL!
work = QuantumRegister(NUM_BITS-1, name="a")
y = QuantumRegister(1, name="y")
x_meas = ClassicalRegister(NUM_BITS, name="m")


# y ^= (x == k)
def u_omega_eq(k):
    qc = QuantumCircuit(x, work, y)

    for i in range(NUM_BITS):
        if k & (1 << i):
            pass
        else:
            qc.x(x[i])

    qc.ccx(x[0], x[1], work[0])
    for i in range(2, NUM_BITS-1):
        qc.ccx(x[i], work[i-2], work[i-1])
    qc.ccx(x[-1], work[NUM_BITS-3], y)
    for i in range(NUM_BITS-2, 1, -1):
        qc.ccx(x[i], work[i-2], work[i-1])
    qc.ccx(x[0], x[1], work[0])
    

    for i in range(NUM_BITS):
        if k & (1 << i):
            pass
        else:
            qc.x(x[i])
    return qc

# y ^= (x < k)
def u_omega_cmp(k):
    qc = QuantumCircuit(x, work, y)
    cmp = FixedValueComparator(NUM_BITS, value=k, geq=False)
    cmp.build(qc, [x[i] for i in range(NUM_BITS)] + [y[0]], work)
    return qc

def a():
    #qc = QuantumCircuit(x)
    #qc.h(x[0])
    #qc.h(x[2])
    #return qc

    a = [0]*(2**NUM_BITS)
    for s in S:
        if s > (2**NUM_BITS):
            pass
        a[s] = 1
    a = [float(i) /np.sqrt(sum(a))  for i in a]
    
    
    
    qc = QuantumCircuit(x) 
    qc.initialize(a,[x[i] for i in range(NUM_BITS)]) 
    dag = circuit_to_dag(qc)
    pass_ = Unroller(basis=['u3', 'cx', 'id'])
    unrolled_dag = pass_.run(dag)
    unrolled_circuit = dag_to_circuit(unrolled_dag)
    import re
    qc_without_reset = re.sub("reset.*\n", "", unrolled_circuit.qasm())
      

   # print ( qc_without_reset) 
    qc = QuantumCircuit.from_qasm_str(qc_without_reset)
    
    return qc 

def a_inv():
    return a().inverse() 

def diffusion():
    qc = QuantumCircuit(x)
    qc += a_inv()
    qc += u_omega_eq(0)
    qc += a()
    return qc

def grover_iteration(u_omega):
    qc = QuantumCircuit()
    qc += u_omega
    qc.barrier()
    qc += diffusion()
    return qc


In [8]:
# TODO: wrong for comparison
THETA = 2*np.arcsin(1/np.sqrt(N))
ITERS = (np.pi/2 - THETA/2) / THETA

print(ITERS)
ITERS = int(np.round(ITERS))
print(ITERS)


num_grover_iters = {0, ITERS}
k = 1
while k <= ITERS:
    num_grover_iters.add(k)
    k *= 2   
#num_grover_iters = {0}
print(num_grover_iters)

def find_k(k):
    shots = 1
    
    qc = QuantumCircuit(x,work,y,x_meas)
    qc += a()
    qc.x(y)
    qc.h(y)
    for _ in range(ITERS):
        qc += grover_iteration(u_omega_eq(k))
    qc.measure(x, x_meas)
    result = execute(qc, backend, shots=shots, seed_simulator=int(np.uint32(100*time.time()))).result()
    #print_statevector(result.get_statevector())
    counts = result.get_counts()
    k_bin = ("{:0%db}"%NUM_BITS).format(k)
    return k_bin in counts

def find_min(iters):
    working_min = 1<<NUM_BITS
    
    for _ in range(iters):
        experiments = []
        for k in num_grover_iters:
            qc = QuantumCircuit(x,work,y,x_meas)
            qc += a()
            qc.x(y)
            qc.h(y)
            for _ in range(k):
                qc += grover_iteration(u_omega_cmp(working_min))
            qc.measure(x, x_meas)
            experiments.append(qc)

        result = execute(experiments, backend, shots=1, seed_simulator=int(np.uint32(100*time.time()))).result()
        for i in range(len(experiments)):
            counts = result.get_counts(i)
            print(counts)
            working_min = min(working_min, min([int(k, 2) for k in counts if counts[k] > 0]))
        print(working_min)
    return working_min

# paper talks of 22.5 iterations
print('min', S, "=", find_min(5))

# this works most of the time
print(17, "in", S, ":", find_k(17))

1.9410157268268091
2
{0, 1, 2}
{'01101': 1}
{'01101': 1}
{'01101': 1}
13
{'10011': 1}
{'01010': 1}
{'10011': 1}
10
{'01101': 1}
{'01101': 1}
{'01101': 1}
10
{'01101': 1}
{'01101': 1}
{'01101': 1}
10
{'10011': 1}
{'10011': 1}
{'10011': 1}
10
min [10, 11, 12, 13, 14, 15, 16, 17, 18, 19] = 10
17 in [10, 11, 12, 13, 14, 15, 16, 17, 18, 19] : True


In [5]:
def print_statevector(statevector):
    for i in range(len(statevector)):
        x_val = i & ((1<<NUM_BITS)-1)
        y_val = i >> (NUM_BITS + NUM_BITS-1)
        if np.abs(statevector[i]) > 1e-8:
            print("{:.3f} |{}>|{}>".format(statevector[i], x_val, y_val))

In [ ]:
qc = QuantumCircuit(x, work, y, x_meas)
qc += a()
qc.x(y)
qc.h(y)
qc += grover_iteration(u_omega_eq(16))
result = execute(qc, simulator).result()
print_statevector(result.get_statevector())

qc.draw(output='mpl')